# Varying Final Probabilities

In [2]:
def generate_WFST_final_probability(n, lex, weights_final, weight_fwd, weight_self, original=False):
    """ generate a HMM to recognise any single word sequence for words in the lexicon
    
    Args:
        n (int): states per phone HMM
        original (bool): True/False - origianl/optimized lexicon
        weight_fwd (int): weight value
        weight_self (int): weight value of self node
        weight_final (dict): word -> probability of final state
    Returns:
        the constructed WFST
    
    """
    
    f = fst.Fst('log')
    none_weight = fst.Weight('log', -math.log(1))

    lex = parse_lexicon(lex, original)
    
    word_table, phone_table, state_table = generate_symbols_table(lex,3)
    output_table = generate_output_table(word_table, phone_table)

    # create a single start state
    start_state = f.add_state()
    f.set_start(start_state)
    
    for word, phone_list in lex.items():
        for phones in phone_list:
            initial_state = f.add_state()
            f.add_arc(start_state, fst.Arc(0, output_table.find(word), none_weight, initial_state))
            current_state = initial_state
            
            for phone in phones:
                current_state = generate_phone_wfst(f, current_state, phone, n, state_table, output_table, weight_fwd, weight_self)
        
            f.set_final(current_state)
            f.add_arc(current_state, fst.Arc(0, 0, none_weight, start_state))
        
        # final word state should be current state
        prob = weights_final[word]
        weight = fst.Weight('log', -math.log(prob))
        f.set_final(current_state, weight)
        
        
    f.set_input_symbols(state_table)
    f.set_output_symbols(output_table)
    return f, word_table 